# XAI Pipeline for Skin Cancer Classification
This notebook loads the trained Xception model, applies there XAI methods (Grad-CAM, SHAP, Influence Functions), and exports the results in a format ready for the LLM integration pipeline.

## Import Libraries

In [ ]:
# --- 1. Setup and Initialization ---
import sys
import os
project_root = os.path.abspath(os.path.join(os.getcwd(), os.pardir))
if project_root not in sys.path:
    sys.path.append(project_root)
    print(f"Project root added to path: {project_root}")

import torch
import torch.nn as nn
import torch.nn.functional as F
from torchvision import transforms
from torch.utils.data import Dataset, DataLoader, TensorDataset
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from captum.attr import LayerGradCam, Occlusion, IntegratedGradients
from captum.attr import visualization as viz
from sklearn.model_selection import train_test_split
from tqdm import tqdm

# Import the model architecture from our central definition file
from src.model_def import SkinCancerViT

print("Libraries imported successfully.")

## Define Project Structure and File Paths

In [ ]:
# --- 2. Define Project Structure and File Paths ---

BASE_DIR = project_root
MODEL_PATH = os.path.join(BASE_DIR, 'models', 'temp', 'ViT-class_weights.pth')
SAMPLE_IMAGE_PATH = os.path.join(BASE_DIR, 'user_inputs', 'user_sample1.jpg')
PROCESSED_TRAIN_DATA_PATH = os.path.join(BASE_DIR, 'data', 'processed', 'ViT', 'train_dataset.pt')
TRAIN_SPLIT_PATH = os.path.join(BASE_DIR, 'data', 'processed', 'train_split.csv')
OUTPUT_DIR = os.path.join(BASE_DIR, 'results', 'xai_output')
os.makedirs(OUTPUT_DIR, exist_ok=True)
print(f"Project Base Directory: {BASE_DIR}")
print(f"Using Model Path: {MODEL_PATH}")

## Model Definition and Loading

In [ ]:
# --- 3. Model Definition and Loading (Final, Corrected Version) ---
print("\nLoading model...")

# Instantiate the model from our central definition file
model = SkinCancerViT(num_classes=2)
try:
    model.load_state_dict(torch.load(MODEL_PATH))
    model.eval()
    print(f"Successfully loaded trained model weights.")
except Exception as e:
    print(f"An error occurred while loading the model state_dict: {e}")

## Data Loading and Preprocessing

In [ ]:
# --- 4. Data Loading and Preprocessing ---

print("\nLoading and preprocessing sample image...")

# The model expects 224x224 images
preprocess_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])
try:
    input_image = Image.open(SAMPLE_IMAGE_PATH).convert('RGB')
    input_tensor = preprocess_transform(input_image).unsqueeze(0)
    print(f"Successfully loaded sample image: {SAMPLE_IMAGE_PATH}")
except FileNotFoundError:
    print(f"ERROR: Sample image not found at '{SAMPLE_IMAGE_PATH}'")

## XAI Pipeline Implementation

In [ ]:
# --- 5. XAI Pipeline Implementation ---

print("\n--- Running XAI Pipeline ---")

# Get model prediction
logits = model(input_tensor).logits
output_softmax = F.softmax(logits, dim=1)
prediction_score, pred_label_idx = torch.topk(output_softmax, 1)
predicted_class_index = pred_label_idx.squeeze().item()
class_names = ['Benign', 'Malignant']
print(f"Model Prediction: Class '{class_names[predicted_class_index]}' with confidence {prediction_score.item():.2%}")

### Grad-CAM Implementation

In [ ]:
# --- 5a. Grad-CAM Implementation ---
print("\n--- Generating Grad-CAM ---")

# We target the initial convolutional layer that processes the image patches.
# This layer is compatible with the standard Grad-CAM algorithm.
grad_cam_layer = model.vit_model.vit.embeddings.patch_embeddings.projection

def model_forward_wrapper(input_tensor):
    """Wrapper to return only the logits tensor from the model's output."""
    return model(input_tensor).logits

layer_gc = LayerGradCam(model_forward_wrapper, grad_cam_layer)
attribution_gc = layer_gc.attribute(input_tensor, target=predicted_class_index)

# The output from the convolutional layer is already a 2D feature map,
# so we can resize it directly without token reshaping.
heatmap = F.interpolate(attribution_gc, size=input_image.size, mode='bilinear', align_corners=False)
heatmap = heatmap.squeeze().cpu().detach().numpy()

# Plotting and saving the Grad-CAM image
fig, ax = plt.subplots(figsize=(8, 8))
ax.imshow(input_image)
ax.imshow(heatmap, cmap='jet', alpha=0.5)
ax.axis('off')

grad_cam_output_path = os.path.join(OUTPUT_DIR, 'user_sample1_xai_gradcam.png')
plt.savefig(grad_cam_output_path, bbox_inches='tight', pad_inches=0)
plt.show()
print(f"Grad-CAM image saved to: {grad_cam_output_path}")

### SHAP

In [ ]:
# --- 5b. SHAP ---
print("\n--- Generating SHAP ---")

occlusion = Occlusion(model_forward_wrapper) # Use the wrapper here too for consistency
attribution_shap = occlusion.attribute(input_tensor, strides=(3, 16, 16), target=predicted_class_index, sliding_window_shapes=(3, 32, 32), baselines=0)

fig_shap, ax_shap = plt.subplots(figsize=(8, 8))
ax_shap.axis('off')
viz.visualize_image_attr(np.transpose(attribution_shap.squeeze().cpu().detach().numpy(), (1,2,0)), np.array(input_image.resize((224,224))), method="blended_heat_map", sign="all", show_colorbar=False, plt_fig_axis=(fig_shap, ax_shap))
shap_output_path = os.path.join(OUTPUT_DIR, 'user_sample1_xai_shap.png')
fig_shap.savefig(shap_output_path, bbox_inches='tight', pad_inches=0)
plt.show()
print(f"SHAP image saved to: {shap_output_path}")

### Influence Functions

In [ ]:
# --- 5c. Influence Functions ---
print("\n--- Generating Influence Functions ---")

def get_gradient_and_prediction(model, data_tensor, target, loss_fn):
    model.zero_grad()
    logits = model(data_tensor).logits
    _, pred_idx = torch.max(logits.data, 1)
    loss = loss_fn(logits, target)
    loss.backward()
    grad = model.vit_model.classifier.weight.grad.detach().clone()
    return grad.flatten(), pred_idx.item()

def calculate_real_influence(model, train_loader, test_tensor, test_target, device):
    model.to(device)
    model.eval()
    test_tensor = test_tensor.to(device)
    test_target = torch.tensor([test_target]).to(device)
    loss_fn = nn.CrossEntropyLoss()
    print("Calculating gradient for the test image...")
    test_grad, _ = get_gradient_and_prediction(model, test_tensor, test_target, loss_fn)
    results = []
    print("Iterating through the training dataset...")
    for train_imgs, train_labels in tqdm(train_loader):
        train_imgs, train_labels = train_imgs.to(device), train_labels.to(device)
        for i in range(len(train_imgs)):
            train_grad, train_pred = get_gradient_and_prediction(model, train_imgs[i].unsqueeze(0), train_labels[i].unsqueeze(0), loss_fn)
            influence_score = torch.dot(test_grad, train_grad).item()
            results.append({'score': influence_score, 'prediction': train_pred})
    return results

try:
    print(f"Loading processed training data from: {PROCESSED_TRAIN_DATA_PATH}")
    processed_data = torch.load(PROCESSED_TRAIN_DATA_PATH)
    train_images, train_labels = processed_data['images'], processed_data['labels']
    train_df = pd.read_csv(TRAIN_SPLIT_PATH)
    train_filenames = train_df['DDI_file'].tolist()
    train_dataset = TensorDataset(train_images, train_labels)
    train_loader = DataLoader(train_dataset, batch_size=32)
    
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    influence_results = calculate_real_influence(model, train_loader, input_tensor, predicted_class_index, device)
    
    report_data = [{'case_id': train_filenames[i].split('.')[0], 'influence_score': r['score'], 'ground_truth': train_labels[i].item(), 'prediction': r['prediction']} for i, r in enumerate(influence_results)]
    report_df = pd.DataFrame(report_data)
    report_df['abs_influence'] = report_df['influence_score'].abs()
    report_df = report_df.sort_values(by='abs_influence', ascending=False).drop(columns='abs_influence')
    final_report_df = report_df.head(100)

    influence_output_path = os.path.join(OUTPUT_DIR, 'user_sample1_influence_function.csv')
    final_report_df.to_csv(influence_output_path, index=False)
    print(f"\nSuccessfully saved {len(final_report_df)} real influence scores to: {influence_output_path}")
    
except FileNotFoundError as e:
    print(f"ERROR: A required data file was not found. Please ensure notebooks 01 and 02 have been run. Details: {e}")

print("\n\n--- XAI Pipeline Execution Complete ---")